In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle

In [30]:
dir = './pre_final_4p5M.pkl'
df = pd.read_pickle(dir)
df.head()

,Dst Port,Protocol,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Std,Flow Byts/s,Fwd IAT Tot,...,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Std,Idle Min,Label
0,49574,6,84.0,84.0,0.0,0.0,0.0,0.0,857142.857143,0.0,...,501,0,32,0.0,0.0,0.0,0.0,0.0,1.677225e+15,DDOS
1,42317,6,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0,0,40,0.0,0.0,0.0,0.0,0.0,1.677225e+15,DDOS
2,17818,6,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0,0,40,0.0,0.0,0.0,0.0,0.0,1.677225e+15,DDOS
3,10950,6,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0,0,40,0.0,0.0,0.0,0.0,0.0,1.677225e+15,DDOS
4,47594,6,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0,0,40,0.0,0.0,0.0,0.0,0.0,1.677225e+15,DDOS


In [31]:
print(df.shape)
n = 100

(4477942, 54)


In [32]:
print(df.columns)

Index(['Dst Port', 'Protocol', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Fwd IAT Tot', 'Fwd IAT Mean',
       'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot',
       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
       'Fwd PSH Flags', 'Fwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s',
       'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'ECE Flag Cnt', 'Down/Up Ratio',
       'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Subflow Fwd Pkts',
       'Subflow Fwd Byts', 'Subflow Bwd Byts', 'Init Fwd Win Byts',
       'Init Bwd Win Byts', 'Fwd Act Data Pkts', 'Fwd Seg Size Min',
       'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Std',
       'Idle Min', 'Label'],
      dtyp

In [33]:
print(len(df))
ddos = df[df['Label'] == 'DDOS']
ddos = ddos[:len(ddos) - (len(ddos) % n)]
ddos.drop(['Label'], axis=1, inplace=True)
print(len(ddos))

password_attack = df[df['Label'] == 'Password_attack']
password_attack = password_attack[:len(password_attack) - (len(password_attack) % n)]
password_attack.drop(['Label'], axis=1, inplace=True)
print(len(password_attack))

port_scan = df[df['Label'] == 'Port_scan']
port_scan = port_scan[:len(port_scan) - (len(port_scan) % n)]
port_scan.drop(['Label'], axis=1, inplace=True)
print(len(port_scan))

benign = df[df['Label'] == 'Benign']
benign = benign[:len(benign)- (len(benign) % n)]
benign.drop(['Label'], axis=1, inplace=True)
print(len(benign))

df.drop(['Label'], axis=1, inplace=True)

4477942
999900
988400
999900
1489400


In [34]:
# data_reshape = df.groupby(np.arange(len(df)) // n, axis = 0).apply(lambda x: x.reset_index(drop=True)).unstack()
data_ddos = np.reshape(ddos.values, (len(ddos) // n, n, ddos.shape[1]))
data_password_attack = np.reshape(password_attack.values, (len(password_attack) // n, n, password_attack.shape[1]))
data_port_scan = np.reshape(port_scan.values, (len(port_scan) // n, n, port_scan.shape[1]))
data_benign = np.reshape(benign.values, (len(benign) // n, n, benign.shape[1]))

In [35]:
print(data_ddos.shape)
print(data_password_attack.shape)
print(data_port_scan.shape)
print(data_benign.shape)

(9999, 100, 53)
(9884, 100, 53)
(9999, 100, 53)
(14894, 100, 53)


In [36]:
labels_ddos = np.zeros(len(ddos)//100).reshape(-1,1)
labels_password_attack = np.full(len(password_attack)//100,1).reshape(-1,1)
labels_port_scan = np.full(len(port_scan)//100,2).reshape(-1,1) 
labels_benign = np.full(len(benign)//100,3).reshape(-1,1)
print(len(labels_ddos))
print(len(labels_password_attack))
print(len(labels_port_scan))
print(len(labels_benign))
labels = np.concatenate((labels_ddos, labels_password_attack, labels_port_scan, labels_benign), axis=0)
labels = pd.DataFrame(labels)

9999
9884
9999
14894


In [37]:
data = np.concatenate((data_ddos, data_password_attack, data_port_scan, data_benign), axis=0)
# save data
pickle.dump(data, open(f'./reshape_df/data.pkl', 'wb'))
labels.to_pickle('./reshape_df/labels.pkl')

ValueError: Must pass 2-d input. shape=(44776, 100, 53)

In [ ]:
# test
data = pickle.load(open('./reshape_df/data.pkl', 'rb'))
labels = pd.read_pickle('./reshape_df/labels.pkl')
print(data.shape)
print(labels.shape)

(44776, 100, 53)
(44776, 1)
